In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf

In [ ]:
%pwd

'/content'

In [ ]:
%ls

sample_data/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/infectious biomarkers.xlsx')

In [ ]:
df.head()

,Biomarker,Function,Range,Associated Diseases
0,C-reactive protein (CRP),Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea..."
1,White blood cell count (WBC),General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias"
2,Procalcitonin (PCT),Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis
3,Interleukins (IL),Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer"
4,Viral load,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV


In [ ]:
df[df.duplicated()]

,Biomarker,Function,Range,Associated Diseases
45,Antibiotic resistance biomarkers,Beta-lactamase enzymes: Indicate resistance to...,Varies,Antibiotic-resistant bacterial infections
69,Prostaglandin E2 (PGE2),Elevated levels associated with inflammatory c...,"25-1,000 pg/mL","Inflammatory Conditions, Infections"
85,Sepsis Biomarkers,Soluble Triggering Receptor Expressed on Myelo...,"sTREM-1: Varies, ProADM: Varies",Sepsis
88,Prostaglandin E2 (PGE2),Elevated levels associated with inflammatory c...,"25-1,000 pg/mL","Inflammatory Conditions, Infections"
122,Interferon-Gamma Release Assays (IGRAs),Blood tests detect the release of interferon-g...,Varies,Tuberculosis Infections
125,Pro-adrenomedullin (ProADM),A precursor molecule associated with systemic ...,Varies,Sepsis


In [ ]:
df.loc[df['Biomarker']  == "Antibiotic resistance biomarkers"]

,Biomarker,Function,Range,Associated Diseases
35,Antibiotic resistance biomarkers,Beta-lactamase enzymes: Indicate resistance to...,Varies,Antibiotic-resistant bacterial infections
45,Antibiotic resistance biomarkers,Beta-lactamase enzymes: Indicate resistance to...,Varies,Antibiotic-resistant bacterial infections
55,Antibiotic resistance biomarkers,Beta-lactamase enzymes: Indicate resistance to...,Varies,Antibiotic-Resistant Bacterial Infections


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(216, 4)

In [ ]:
df1 = df.copy()

In [ ]:
from datetime import datetime
def get_type(ele):
    if type(ele) == datetime:
        ele = str(ele)
    return ele

df1['Range'] = df1['Range'].apply(get_type)

In [ ]:
df1['Range']

0               0.3 to 1.0 mg/dl
1        4,500 to 11,000 WBCs/μL
2                    < 0.1 ng/mL
3         0.007 ng/ml or 7 pg/ml
4                < 200 copies/mL
                 ...            
217    IgM: Positive or Negative
218                       Varies
219                       Varies
220         Positive or Negative
221                          NaN
Name: Range, Length: 216, dtype: object

In [ ]:
df1 = df1.dropna()

In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer

dfpn = pd.DataFrame()
dfv = pd.DataFrame()
dfc = pd.DataFrame()
dfvlms = pd.DataFrame(columns=df1.columns)  # Initialize dfvlms with columns from df1

volumes = ['μl' ,'ml' ,'dl', 'l']
for lines in df1['Range']:
    pattern = r'\w+|\.'
    tokenizer = RegexpTokenizer(pattern)
    words = tokenizer.tokenize(lines)
    updated_words = [word.lower() for word in words]
    result_string = ' '.join(updated_words)

    if 'positive' in updated_words or 'negative' in updated_words:
        dfpn = pd.concat([dfpn,df1.loc[df1['Range'] == lines]])
    elif 'varies' in updated_words:
        dfv = pd.concat([dfv ,df1.loc[df1['Range'] == lines]])
    elif updated_words[-1] in volumes:
        dfvlms = pd.concat([dfvlms ,df1.loc[df1['Range'] == lines]])
    else:
        dfc = pd.concat([dfc , df1.loc[df1['Range'] == lines]])




In [ ]:
dfc = dfc.drop_duplicates()

In [ ]:
dfc.shape

(17, 4)

In [ ]:
dfvlms = dfvlms.drop_duplicates()

In [ ]:
dfvlms.shape

(37, 4)

In [ ]:

dfv = dfv.drop_duplicates()

In [ ]:
dfv.shape

(114, 4)

In [ ]:
dfpn = dfpn.drop_duplicates()

In [ ]:
dfpn.shape

(47, 4)

In [ ]:
dfpn.head()

,Biomarker,Function,Range,Associated Diseases
5,Antibody titers,Measure specific antibodies produced in respon...,Zero or negative,"Measles, Mumps, Rubella, Varicella zoster viru..."
30,Lyme disease biomarkers,Borrelia burgdorferi antibodies: Detects antib...,Borrelia burgdorferi antibodies: < 0.9 = negat...,Lyme disease infection
52,Fungal biomarkers,β-D-glucan: Detects invasive fungal infections...,"β-D-glucan: 30 to 59 pg/mL Negative, 60-80 pg/...",Invasive Fungal Infections
77,Lyme Disease Biomarkers,Borrelia burgdorferi Antibodies: Detects antib...,Borrelia burgdorferi Antibodies: < 0.9 = negat...,Lyme Disease
89,β-D-Glucan,A polysaccharide present in the cell wall of m...,"Negative: < 60 pg/mL, Indeterminate: 60-80 pg/...","Invasive Aspergillosis (IA), Invasive Candidia..."


In [ ]:
dfvlms.head()

,Biomarker,Function,Range,Associated Diseases
0,C-reactive protein (CRP),Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea..."
1,White blood cell count (WBC),General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias"
2,Procalcitonin (PCT),Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis
3,Interleukins (IL),Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer"
4,Viral load,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV


In [ ]:
df2 = df.copy()

In [ ]:
df2.head()

,Biomarker,Function,Range,Associated Diseases
0,C-reactive protein (CRP),Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea..."
1,White blood cell count (WBC),General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias"
2,Procalcitonin (PCT),Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis
3,Interleukins (IL),Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer"
4,Viral load,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV


In [ ]:

from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()

In [ ]:
df2['Biomarker'] = l.fit_transform(df2['Biomarker'])

In [ ]:
df2['Biomarker']

0       20
1      190
2      142
3       97
4      187
      ... 
217     53
218     81
219     98
220     87
221     69
Name: Biomarker, Length: 216, dtype: int64

In [ ]:
df2.head()

,Biomarker,Function,Range,Associated Diseases
0,20,Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea..."
1,190,General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias"
2,142,Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis
3,97,Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer"
4,187,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV


In [ ]:
diseases = df['Associated Diseases']

In [ ]:
diseases

0      Rheumatoid arthritis, Inflammatory bowel disea...
1                   Proliferative disorders, Leukopenias
2                                          Severe sepsis
3                 Diabetes, Rheumatoid arthritis, Cancer
4                                                    HIV
                             ...                        
217                   Epstein-Barr Virus (EBV) Infection
218                      Cytomegalovirus (CMV) Infection
219     Progressive Multifocal Leukoencephalopathy (PML)
220    Human T-cell Leukemia Virus Type 1 (HTLV-1) In...
221                                                  NaN
Name: Associated Diseases, Length: 216, dtype: object

In [ ]:
def typecast(ele):
    if type(ele)!=str:
        str(ele)
    return str(ele)

In [ ]:
diseases = diseases.apply(typecast)

In [ ]:
diseases.dtype

dtype('O')

In [ ]:
lists =[]
def make_list(ele):
    values = ele.split(",")
    for i in values:
        lists.append(i)


In [ ]:
diseases.apply(make_list)

0      None
1      None
2      None
3      None
4      None
       ... 
217    None
218    None
219    None
220    None
221    None
Name: Associated Diseases, Length: 216, dtype: object

In [ ]:
lists

['Rheumatoid arthritis',
 ' Inflammatory bowel disease',
 ' Influenza virus',
 ' Adenovirus',
 'Proliferative disorders',
 ' Leukopenias',
 'Severe sepsis',
 'Diabetes',
 ' Rheumatoid arthritis',
 ' Cancer',
 'HIV',
 'Measles',
 ' Mumps',
 ' Rubella',
 ' Varicella zoster virus',
 ' Hepatitis A',
 ' Hepatitis B',
 ' Tetanus',
 ' Diphtheria',
 ' COVID-19',
 'Influenza viruses',
 ' Respiratory syncytial virus (RSV)',
 'Sepsis',
 'SARS-CoV-2 (COVID-19)',
 'Tissue damage',
 ' Muscular dystrophy',
 'Blood clotting problems',
 'Arthritis',
 ' Vasculitis',
 ' Inflammatory bowel disease',
 'Fatty liver disease',
 ' Toxic Hepatitis',
 ' Viral hepatitis (A',
 ' B or C)',
 ' Hemochromatosis',
 " Wilson's disease",
 ' Alpha-1 antitrypsin deficiency',
 ' Primary biliary cholangitis (PBC)',
 'Skeletal muscle',
 ' Heart or brain damage or degeneration',
 'Bacteremia',
 ' Sepsis',
 'Myocardial infarction',
 ' Angina',
 ' Heart failure',
 'Colorectal cancer',
 ' Ulcer',
 ' Polyps',
 'Lyme disease',
 ' S

In [ ]:
modified_list = [item.strip().lower() for item in lists]


In [ ]:
modified_list

['rheumatoid arthritis',
 'inflammatory bowel disease',
 'influenza virus',
 'adenovirus',
 'proliferative disorders',
 'leukopenias',
 'severe sepsis',
 'diabetes',
 'rheumatoid arthritis',
 'cancer',
 'hiv',
 'measles',
 'mumps',
 'rubella',
 'varicella zoster virus',
 'hepatitis a',
 'hepatitis b',
 'tetanus',
 'diphtheria',
 'covid-19',
 'influenza viruses',
 'respiratory syncytial virus (rsv)',
 'sepsis',
 'sars-cov-2 (covid-19)',
 'tissue damage',
 'muscular dystrophy',
 'blood clotting problems',
 'arthritis',
 'vasculitis',
 'inflammatory bowel disease',
 'fatty liver disease',
 'toxic hepatitis',
 'viral hepatitis (a',
 'b or c)',
 'hemochromatosis',
 "wilson's disease",
 'alpha-1 antitrypsin deficiency',
 'primary biliary cholangitis (pbc)',
 'skeletal muscle',
 'heart or brain damage or degeneration',
 'bacteremia',
 'sepsis',
 'myocardial infarction',
 'angina',
 'heart failure',
 'colorectal cancer',
 'ulcer',
 'polyps',
 'lyme disease',
 'syphilis',
 'viral infections',
 

In [ ]:
unique_words = set()

for item in modified_list:
    unique_words.add(item.strip().lower())

unique_list = list(unique_words)

print(unique_list)

['parasitic infections', 'toxoplasma gondii infection', 'west nile virus infection', 'or c)', 'syphilis', 'pneumococcal pneumonia', 'coxiella burnetii infection', 'chlamydia infections', 'trichuriasis', 'cytomegalovirus (cmv) infection', 'human t-cell leukemia virus type 1 (htlv-1) infection', 'chlamydia trachomatis infection', 'cancer', 'bacteremia', 'bacterial meningitis', 'rocky mountain spotted fever', 'hiv', 'hiv infection', 'marburg virus disease', 'echinococcosis', 'heart or brain damage or degeneration', 'q fever', 'bk virus infection', 'tuberculosis infections', 'hepatitis b virus infection', 'acid-base disturbances', 'pneumonia infections', 'tissue damage', 'human immunodeficiency virus type 2 (hiv-2) infection', 'tuberculosis', 'mumps virus infection', 'influenza infection', 'cryptosporidiosis', 'mycoplasma pneumoniae infection', 'entamoebiasis', 'strongyloidiasis', 'severe infections', 'middle east respiratory syndrome (mers)', 'severe sepsis', 'schistosomiasis', 'invasive 

In [ ]:
"tuberculosis infections" in unique_list

True

In [ ]:
disease_set = set(unique_list)

In [ ]:
len(disease_set)

194

In [ ]:
disease_set

{'acid-base disturbances',
 'adenovirus',
 'adenovirus infection',
 'aids',
 'alpha-1 antitrypsin deficiency',
 'amebiasis',
 'angina',
 'anthrax',
 'antibiotic-resistant bacterial infections',
 'arthritis',
 'ascariasis',
 'aspergillosis',
 'autoimmune hepatitis',
 'b',
 'b or c)',
 'babesiosis',
 'bacteremia',
 'bacterial infections',
 'bacterial meningitis',
 'bacterial or fungal infections',
 'bk virus infection',
 'blastomycosis',
 'blood clotting problems',
 'brucellosis',
 'campylobacter infection',
 'cancer',
 'chagas disease',
 'chikungunya virus infection',
 'chlamydia and gonorrhea infections',
 'chlamydia infections',
 'chlamydia pneumoniae infection',
 'chlamydia trachomatis infection',
 'cholera',
 'clonorchiasis',
 'clostridium difficile infection',
 'clostridium difficile infection (cdi)',
 'coccidioidomycosis',
 'colorectal cancer',
 'covid-19',
 'covid-19)',
 'coxiella burnetii infection',
 'coxsackievirus infection',
 'cryptococcal meningitis',
 'cryptosporidiosis',


In [ ]:
diseases_data = pd.DataFrame(disease_set ,columns=['Disease'])

In [ ]:
diseases_data.head()

,Disease
0,parasitic infections
1,toxoplasma gondii infection
2,west nile virus infection
3,or c)
4,syphilis


In [ ]:
diseases_data.loc[diseases_data["Disease"] == 'rheumatoid arthritis']

,Disease
156,rheumatoid arthritis


In [ ]:
diseases_data.loc[diseases_data["Disease"] == 'rheumatoid arthritis']

,Disease
156,rheumatoid arthritis


In [ ]:
from sklearn.preprocessing import LabelEncoder
l1 = LabelEncoder()

In [ ]:
diseases_data['Encode'] = l1.fit_transform(diseases_data['Disease'])

In [ ]:
diseases_data['Encode']

0      135
1      173
2      191
3      132
4      167
      ... 
189     59
190    147
191    157
192     60
193     44
Name: Encode, Length: 194, dtype: int64

In [ ]:
diseases_data.head()

,Disease,Encode
0,parasitic infections,135
1,toxoplasma gondii infection,173
2,west nile virus infection,191
3,or c),132
4,syphilis,167


In [ ]:
c = diseases_data.loc[diseases_data['Disease'] == "arthritis"]

In [ ]:
c

,Disease,Encode
67,arthritis,9


In [ ]:
int(c['Encode'])

9

In [ ]:

def transform(ele):
    l=[]
    val = ele.split(",")
    for disease in val:
        disease = disease.strip().lower()
        code = diseases_data.loc[diseases_data['Disease'] == disease]
        l.append(int(code['Encode']))
    return l


# New Section

In [ ]:
df2['Output'] = df1['Associated Diseases'].apply(transform)

In [ ]:
diseases_data.loc[diseases_data["Disease"] == "severe sepsis"]

,Disease,Encode
38,severe sepsis,162


In [ ]:
df2.head()

,Biomarker,Function,Range,Associated Diseases,Output
0,20,Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea...","[148, 97, 102, 1]"
1,190,General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias","[143, 113]"
2,142,Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis,[162]
3,97,Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer","[48, 148, 25]"
4,187,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV,[82]


In [ ]:
df2.loc[df2['Associated Diseases'] == 'Sepsis']

,Biomarker,Function,Range,Associated Diseases,Output
7,15,Incubate blood sample to detect bacteria or fu...,Adults: 10-20 mL per culture set; Pediatric pa...,Sepsis,[157]
33,158,sTREM-1 and ProADM: Indicate bacterial infecti...,"sTREM-1: Varies, ProADM: Varies",Sepsis,[157]
37,158,Procalcitonin (PCT): Elevated levels can indic...,"PCT: < 0.05 ng/mL, Lactate: < 2 mmol/L",Sepsis,[157]
43,158,Soluble triggering receptor expressed on myelo...,"sTREM-1: Varies, ProADM: Varies",Sepsis,[157]
66,157,Soluble Triggering Receptor Expressed on Myelo...,"sTREM-1: Varies, ProADM: Varies",Sepsis,[157]
94,160,Increased levels in blood can indicate bacteri...,Varies,Sepsis,[157]
95,140,A precursor molecule associated with systemic ...,Varies,Sepsis,[157]
120,96,A pro-inflammatory cytokine produced in respon...,75 +/- 15 pg/ml,Sepsis,[157]


In [ ]:
df2.loc[df2['Associated Diseases'] == 'Anthrax']

,Biomarker,Function,Range,Associated Diseases,Output
213,1,Elevated levels of Anthrax Lethal Factor can i...,Varies,Anthrax,[7]


In [ ]:
df3 = df2.copy()

In [ ]:
delist = diseases_data["Encode"]

In [ ]:
delist = set(delist)

In [ ]:
len(delist)

194

In [ ]:
df3.head()

,Biomarker,Function,Range,Associated Diseases,Output
0,20,Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea...","[148, 97, 102, 1]"
1,190,General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias","[143, 113]"
2,142,Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis,[162]
3,97,Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer","[48, 148, 25]"
4,187,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV,[82]


In [ ]:
df3.to_csv("new_dat.csv")

In [ ]:
df3.dropna(inplace = True)

In [ ]:
x = df3.iterrows()
print(x)

<generator object DataFrame.iterrows at 0x79074c687ca0>


In [ ]:
import pandas as pd
import numpy as np


for ele in delist:
  df3[ele] = [0]*215


<ipython-input-70-2171bf974abb>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3[ele] = [0]*215
<ipython-input-70-2171bf974abb>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3[ele] = [0]*215
<ipython-input-70-2171bf974abb>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3[ele] = [0]*215
<

In [ ]:
df3.head()

,Biomarker,Function,Range,Associated Diseases,Output,0,1,2,3,4,...,184,185,186,187,188,189,190,191,192,193
0,20,Acute-phase protein produced by the liver in r...,0.3 to 1.0 mg/dl,"Rheumatoid arthritis, Inflammatory bowel disea...","[148, 97, 102, 1]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,190,General marker of infection and inflammation. ...,"4,500 to 11,000 WBCs/μL","Proliferative disorders, Leukopenias","[143, 113]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,142,Released during bacterial infections. Elevated...,< 0.1 ng/mL,Severe sepsis,[162],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,97,Pro-inflammatory cytokines released in respons...,0.007 ng/ml or 7 pg/ml,"Diabetes, Rheumatoid arthritis, Cancer","[48, 148, 25]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,187,Amount of virus present in the body. Useful fo...,< 200 copies/mL,HIV,[82],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df4 = df3.copy()

In [ ]:
df4.dropna(inplace =True)

In [ ]:
df4.iloc[: ,4:]

,Output,0,1,2,3,4,5,6,7,8,...,184,185,186,187,188,189,190,191,192,193
0,"[148, 97, 102, 1]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[143, 113]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[162],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[48, 148, 25]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[82],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,[31],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217,[55],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,[45],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,[142],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df4.columns

Index([          'Biomarker',            'Function',               'Range',
       'Associated Diseases',              'Output',                     0,
                           1,                     2,                     3,
                           4,
       ...
                         184,                   185,                   186,
                         187,                   188,                   189,
                         190,                   191,                   192,
                         193],
      dtype='object', length=199)

In [ ]:
array2 = df4.iloc[:,4]

In [ ]:
array2

0      [148, 97, 102, 1]
1             [143, 113]
2                  [162]
3          [48, 148, 25]
4                   [82]
             ...        
216                 [31]
217                 [55]
218                 [45]
219                [142]
220                 [91]
Name: Output, Length: 215, dtype: object

In [ ]:
array2.dtype

dtype('O')

In [ ]:
array2.shape

(215,)

In [ ]:
array2

0      [148, 97, 102, 1]
1             [143, 113]
2                  [162]
3          [48, 148, 25]
4                   [82]
             ...        
216                 [31]
217                 [55]
218                 [45]
219                [142]
220                 [91]
Name: Output, Length: 215, dtype: object

In [ ]:
import numpy as np


In [ ]:
array1 = np.zeros((215 , 195))

In [ ]:
array1[45,0]

0.0

In [ ]:
for i in range(array1.shape[0]):
  for j in array2[i]:
    array1[i,j]=1


KeyError: 45

In [ ]:
array1[0,65]

0.0

In [ ]:
array1[0,187]

0.0

In [ ]:
df5 = pd.DataFrame(array1)

In [ ]:
df5.head()

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df5['Output'] = df4["Output"]

In [ ]:
df5.head()

,0,1,2,3,4,5,6,7,8,9,...,184,185,186,187,188,189,190,191,192,Output
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[148, 97, 102, 1]"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[143, 113]"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[162]
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[48, 148, 25]"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[82]


In [ ]:
df5.shape

(215, 194)

In [ ]:
# Pop the 198th column
op = df5.pop('Output')

# Insert the popped column at the beginning
df5.insert(0, 'Output', op)


In [ ]:
df5.head()

,Output,0,1,2,3,4,5,6,7,8,...,183,184,185,186,187,188,189,190,191,192
0,"[148, 97, 102, 1]",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[143, 113]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[162],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[48, 148, 25]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[82],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df5.loc[4 ,82]

1.0

In [ ]:
df5.loc[: , 0:].astype(int)

In [ ]:
df5.head(37)

,Output,0,1,2,3,4,5,6,7,8,...,183,184,185,186,187,188,189,190,191,192
0,"[148, 97, 102, 1]",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[143, 113]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[162],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[48, 148, 25]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,[82],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"[119, 123, 152, 184, 71, 73, 169, 49, 38]",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,"[103, 146]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,[157],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,[155],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"[170, 126]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df5.tail(10)

,Output,0,1,2,3,4,5,6,7,8,...,183,184,185,186,187,188,189,190,191,192
205,[78],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,[178],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,[87],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208,[191],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209,[116],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
210,[122],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211,[158],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212,[130],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213,[7],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214,[181],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df5.loc[36 , 34]

0

In [ ]:
df5.loc[30 , 115]

0

In [ ]:
df5.to_csv("numericaldisease_data.csv")